In [ ]:
pip install langchain openai chromadb tiktoken unstructured pypdf

In [8]:
import os
import sys

import openai
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

import constants

os.environ["OPENAI_API_KEY"] = constants.APIKEY

# Enable to save to disk & reuse the model (for repeated queries on the same data)
PERSIST = False

query = None
if len(sys.argv) > 1:
  query = sys.argv[1]

if PERSIST and os.path.exists("persist"):
  print("Reusing index...\n")
  vectorstore = Chroma(persist_directory="persist", embedding_function=OpenAIEmbeddings())
  index = VectorStoreIndexWrapper(vectorstore=vectorstore)
else:

  loader = TextLoader("resume.txt") # Use this line if you only need data.txt
  # loader = DirectoryLoader("data/")
  # loader = PyPDFLoader("CV.pdf")


  if PERSIST:
    index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory":"persist"}).from_loaders([loader])
  else:
    index = VectorstoreIndexCreator().from_loaders([loader])

chain = ConversationalRetrievalChain.from_llm(
  llm=ChatOpenAI(model="gpt-3.5-turbo"),
  retriever=index.vectorstore.as_retriever(search_kwargs={"k": 1}),
)

chat_history = []
while True:
  if not query:
    query = input("Prompt: ")
  if query in ['quit', 'q', 'exit']:
    sys.exit()
  result = chain({"question": query, "chat_history": chat_history})
  print("AI:",result['answer'])

  chat_history.append((query, result['answer']))
  query = None

AI: I'm sorry, but it seems like your message got cut off. Could you please provide more information or rephrase your question?
Prompt: quel sont mes diplomes ?
AI: Vous avez obtenu une licence en informatique à l'Université Sorbonne Paris Nord et vous êtes actuellement en train de poursuivre un Master en Data Science (mention Bien) à la même université.
Prompt: quelles sont mes competences sous formes des puces 
AI: - Gestion des données des maladies chroniques (nettoyage, profilage, fusion et génération de rapports)
- Analyse exploratoire et classification de la faillite des entreprises avec l'apprentissage supervisé
- Système décentralisé de transfert de fichier (p2p file sharing)
- Mini compilateur de JavaScript avec ocaml (de .js à .asm)
- Système de sécurité et de surveillance avec la Raspberry pi (IoT)
- Langues étrangères: Anglais (niveau B2), Russe (niveau A1), Arabe (niveau C1)
- Compléter et implémenter les projets de programmation pour les étudiants de la licence informatiq

KeyboardInterrupt: ignored